In [0]:
%skip
drop table aviation_project.gold.fact_flights

In [0]:
%sql
CREATE OR REPLACE TABLE aviation_project.gold.fact_flights AS
WITH cte1 AS (
    SELECT
        s.*,
        a.AIRLINE_SK
    FROM aviation_project.silver.flights s
    LEFT JOIN aviation_project.gold.dim_airlines a
        ON s.AIRLINE = a.IATA_CODE
),
cte2 AS (
    SELECT
        f.*,
        o.AIRPORTS_SK AS ORIGIN_AIRPORT_SK
    FROM cte1 f
    LEFT JOIN aviation_project.gold.dim_airports o
        ON f.ORIGIN_AIRPORT = o.IATA_CODE
),
cte3 AS (
    SELECT
        f.*,
        d.AIRPORTS_SK AS DESTINATION_AIRPORT_SK
    FROM cte2 f
    LEFT JOIN aviation_project.gold.dim_airports d
        ON f.DESTINATION_AIRPORT = d.IATA_CODE
),
cte_date AS (
    SELECT
        f.*,
        dd.DATE_SK,
        MAKE_DATE(f.YEAR, f.MONTH, f.DAY) AS FLIGHT_DATE
    FROM cte3 f
    LEFT JOIN aviation_project.gold.dim_date dd
        ON f.YEAR  = dd.YEAR
       AND f.MONTH = dd.MONTH
       AND f.DAY   = dd.DAY
)
SELECT
    DATE_SK,
    AIRLINE_SK,
    ORIGIN_AIRPORT_SK,
    DESTINATION_AIRPORT_SK,
    DEPARTURE_DELAY,
    ARRIVAL_DELAY,
    TOTAL_DELAY,
    AIR_SYSTEM_DELAY,
    SECURITY_DELAY,
    AIRLINE_DELAY,
    LATE_AIRCRAFT_DELAY,
    WEATHER_DELAY,
    DISTANCE,
    CANCELLED,
    DIVERTED,
    ON_TIME_FLAG,
    FLIGHT_DATE
FROM cte_date

In [0]:
%sql

select * from aviation_project.gold.fact_flights

**Data Quality Checks**

In [0]:
%sql

-- Duplicate records check

SELECT
  AIRLINE_SK,
  ORIGIN_AIRPORT_SK,
  DESTINATION_AIRPORT_SK,
  COUNT(*) AS duplicate_count
FROM aviation_project.gold.fact_flights
GROUP BY
  AIRLINE_SK,
  ORIGIN_AIRPORT_SK,
  DESTINATION_AIRPORT_SK
HAVING COUNT(*) > 1;


In [0]:
%skip

select * from aviation_project.gold.fact_flights
where airline_sk = '1cf587e0bafeb6195595fcbdce170411d52ea40dc4e3582aa9bb272d6a4a01a8'

In [0]:
%sql
-- Check nulls in all the columns in our fact_flights table
SELECT
  SUM(CASE WHEN AIRLINE_SK IS NULL THEN 1 ELSE 0 END) AS null_airline_sk,
  SUM(CASE WHEN DATE_SK IS NULL THEN 1 ELSE 0 END) AS null_date_sk,
  SUM(CASE WHEN FLIGHT_DATE IS NULL THEN 1 ELSE 0 END) AS null_flight_date,
  SUM(CASE WHEN TOTAL_DELAY IS NULL THEN 1 ELSE 0 END) AS null_total_delay
FROM aviation_project.gold.fact_flights;


In [0]:
%sql
-- summary table for invalid delays, distance and cancellation
CREATE OR REPLACE VIEW aviation_project.gold.dq_summary AS
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN TOTAL_DELAY < 0 THEN 1 ELSE 0 END) AS negative_delays,
  SUM(CASE WHEN DISTANCE <= 0 THEN 1 ELSE 0 END) AS invalid_distance,
  SUM(CASE WHEN CANCELLED = 1 AND TOTAL_DELAY > 0 THEN 1 ELSE 0 END) AS cancelled_with_delay
FROM aviation_project.gold.fact_flights;


In [0]:
%sql
-- Flight can depart early from the expected time so negative values is fine in some cases
select * from aviation_project.gold.dq_summary

Referntial Integrity check

In [0]:
%sql
SELECT COUNT(*) AS orphan_airlines
FROM aviation_project.gold.fact_flights f
LEFT JOIN aviation_project.gold.dim_airlines d
  ON f.AIRLINE_SK = d.AIRLINE_SK
WHERE d.AIRLINE_SK IS NULL;


In [0]:
%sql
SELECT COUNT(*) AS orphan_airports
FROM aviation_project.gold.fact_flights f
LEFT JOIN aviation_project.gold.dim_airports a
  ON f.ORIGIN_AIRPORT_SK = a.AIRPORTS_SK
WHERE a.AIRPORTS_SK IS NULL;


In [0]:
%sql
SELECT COUNT(*) AS orphan_dates
FROM aviation_project.gold.fact_flights f
LEFT JOIN aviation_project.gold.dim_date d
  ON f.DATE_SK = d.DATE_SK
WHERE d.DATE_SK IS NULL;
